In [ ]:
%pip install gymnasium==0.29.1
%pip install rl-zoo3==2.2.1
%pip install sb3-contrib==2.2.1
%pip install stable-baselines3==2.2.1
%pip install panda-gym==3.0.7

# %pip install torch==2.2.0
# python 310

# verbose = 0  (출력하지 않음 X)
# verbose = 1  (정보를 상세하게 출력함)
# verbose = 2  (정보를 함축적으로 출력함)

## `라이브러리`

In [ ]:
import panda_gym
import sys
import gymnasium
import numpy as np
from sb3_contrib import TQC
from stable_baselines3.common.logger import configure

## `초기화용`

In [1]:


# gymnasium을 gym으로 인식시키기
sys.modules["gym"] = gymnasium

# 로거 설정
tmp_path = "/tmp/sb3_log/"
new_logger = configure(tmp_path, ["stdout", "csv", "tensorboard"])

# 환경 생성
env = gymnasium.make("PandaPickAndPlace-v3")

# TQC 모델 초기화
model = TQC(policy="MultiInputPolicy", env=env, verbose=1)
model.set_logger(new_logger)

# 초기 경험 수집
initial_steps = 10000  # 필요한 초기 단계 수
obs, _ = env.reset()
for _ in range(initial_steps):
    action = env.action_space.sample()  # 랜덤 행동
    next_obs, reward, done, truncated, info = env.step(action)
    
    # 관찰 값이 리스트나 튜플일 경우 처리
    if isinstance(obs, (list, tuple)):
        obs = np.array(obs)
        next_obs = np.array(next_obs)

    # 관찰 값이 딕셔너리가 아닌 경우 처리 및 추가 키 포함
    if not isinstance(obs, dict):
        obs_dict = {
            'observation': obs.flatten(),
            'achieved_goal': obs.flatten()[:3],  # 필요에 맞게 수정
            'desired_goal': obs.flatten()[3:6]   # 필요에 맞게 수정
        }
        next_obs_dict = {
            'observation': next_obs.flatten(),
            'achieved_goal': next_obs.flatten()[:3],  # 필요에 맞게 수정
            'desired_goal': next_obs.flatten()[3:6]   # 필요에 맞게 수정
        }
    else:
        obs_dict = obs
        next_obs_dict = next_obs
    
    model.replay_buffer.add(obs_dict, next_obs_dict, action, reward, done, [info])
    if done or truncated:
        obs, _ = env.reset()
    else:
        obs = next_obs

# 모델 학습
model.learn(total_timesteps=10000)

# 모델 저장
model.save("ppap")

# 추가 학습
model.learn(total_timesteps=5000)

# 모델 저장
model.save("ppap_updated")


Logging to /tmp/sb3_log/
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 0        |
|    success_rate    | 1        |
| time/              |          |
|    episodes        | 4        |
|    fps             | 571      |
|    time_elapsed    | 0        |
|    total_timesteps | 4        |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 0        |
|    success_rate    | 1        |
| time/              |          |
|    episodes        | 8        |
|    fps             | 294      |
|    time_elapsed    | 0        |
|    total_timesteps | 8        |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 0       

## `업데이트 버전`

In [ ]:


# gymnasium을 gym으로 인식시키기
sys.modules["gym"] = gymnasium

# 로거 설정
tmp_path = "/tmp/sb3_log/"
new_logger = configure(tmp_path, ["stdout", "csv", "tensorboard"])

# 환경 생성
env = gymnasium.make("PandaPickAndPlace-v3")

# 저장된 모델 불러오기
model = TQC.load("ppap_updated", env=env)
model.set_logger(new_logger)

# 모델 학습
model.learn(total_timesteps=10000)

# 모델 저장
model.save("ppap_updated")


In [4]:
from numpngw import write_apng
# gymnasium을 gym으로 인식시키기
sys.modules["gym"] = gymnasium

# 로거 설정
tmp_path = "/tmp/sb3_log/"
new_logger = configure(tmp_path, ["stdout", "csv", "tensorboard"])

# 환경 생성
env = gymnasium.make("PandaPickAndPlace-v3")

# 저장된 모델 불러오기
model = TQC.load("ppap_updated", env=env)
model.set_logger(new_logger)

# 이미지 리스트 초기화
images = []

# 환경 초기화 및 모델 평가
obs, info = env.reset()
for _ in range(1000):  # 1000 스텝 동안 평가
    action, _states = model.predict(obs)
    obs, reward, done, truncated, info = env.step(action)
    
    # 각 스텝에서 이미지를 캡처하여 리스트에 추가
    img = env.render()
    images.append(img)
    
    if done or truncated:
        obs, info = env.reset()

env.close()

# 이미지 리스트를 APNG로 저장
write_apng("testAnim.png", images, delay=200)  # 200 ms 딜레이로 애니메이션 생성


Logging to /tmp/sb3_log/
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [1]:
# import panda_gym
# import sys
# import gymnasium
# sys.modules["gym"] = gymnasium
# # from stable_baselines3 import DDPG
# from sb3_contrib import TQC
# from stable_baselines3.common.logger import configure

# # set up logger
# tmp_path = "/tmp/sb3_log/"
# new_logger = configure(tmp_path, ["stdout", "csv", "tensorboard"])


# env = gymnasium.make("PandaPickAndPlace-v3")

# model = TQC(policy="MultiInputPolicy", env=env, verbose=1)
# model.set_logger(new_logger)
# model.train(1000)
# model.save("ppap")

Logging to /tmp/sb3_log/
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


ValueError: high <= 0